In [1]:
!nvidia-smi

Wed Jul 20 11:55:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# INSTALLS
!pip install lightkurve
!pip install tslearn 
!pip install pytictoc

# IMPORTING NECESSARY LIBRARIES
import lightkurve as lk
import pandas as pd
import numpy as np
from tslearn.utils import to_time_series_dataset
from tslearn.preprocessing import TimeSeriesResampler
from pytictoc import TicToc

# IMPORT AND PROCESSING THE KEPLER ID DATA
finalData = pd.read_csv('finalData.csv')
finalData = finalData[finalData['koi_disposition'].str.contains('CANDIDATE')==False] # removing 'candidate' rows
finalData = finalData.drop_duplicates(subset=['kepid']) # removing duplicates where kepid is repeated ==> CONTROVERSIAL!!! to deal with multiple planets (0, 1, 1+) classification, need raw data
finalData = finalData.replace(['CONFIRMED', 'FALSE POSITIVE'], [1, 0]) # converting 'confirmed' / 'false positive' into 1 / 0
data = finalData
data = data.reset_index(drop=True)

initial_X1 = []
initial_Y1 = []
i = 0 # dummy variable to verify the loop is working

for kepid in data['kepid'].iloc[0:1000]:
  t = TicToc()
  start_time = t.tic()


  try: 
    # DOWNLOADING THE DATA
    KIC = 'KIC ' + str(kepid)
    lcs = lk.search_lightcurve(KIC, author='kepler', cadence='long').download_all()

    # FINDING THE PERIOD, T0 AND DURATION FOR PROCESSING
    row_number = finalData[finalData['kepid'] == kepid].index[0]
    period, t0, duration_hours =  finalData['koi_period'][row_number],  finalData['koi_time0bk'][row_number],  finalData['koi_duration'][row_number]
    
    # PROCESSING THE LIGHTKURVE DATA
    lc_raw = lcs.stitch()
    lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4)
    temp_fold = lc_clean.fold(period, epoch_time=t0)
    fractional_duration = (duration_hours / 24.0) / period
    phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
    transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])
    lc_flat, trend_lc = lc_clean.flatten(return_trend=True, mask=transit_mask)
    lc_fold = lc_flat.fold(period, epoch_time=t0)
    
    # CREATING THE GLOBAL VIEW
    lc_global = lc_fold.bin(time_bin_size=0.005).normalize() - 1
    lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1


    # CONVERTING TO PANDAS DF
    lc_global = lc_global.to_pandas()

    # CREATE LISTS OF X (TIMESERIES DATA) AND Y (LABEL)
    initial_X1.append(lc_global['flux'].tolist())
    initial_Y1.append(finalData['koi_disposition'][row_number])

    print(i)
    i += 1

    end_time = t.toc()
    print(end_time)


  except: 
    pass


# converting initial_X and initial_Y to pandas df
df1 = pd.DataFrame(initial_X1)
df1['target'] = initial_Y1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 247 kB 4.3 MB/s 
     |████████████████████████████████| 4.5 MB 52.9 MB/s 
     |████████████████████████████████| 41 kB 187 kB/s 
     |████████████████████████████████| 98 kB 8.0 MB/s 
     |████████████████████████████████| 832 kB 61.0 MB/s 
     |████████████████████████████████| 48 kB 4.8 MB/s 
     |████████████████████████████████| 4.1 MB 41.4 MB/s 
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11375 sha256=0d1fee7480c5a4ffe8b603a8a53741a211ad310d50f3db86af5215fc0b170be5
  Stored in directory: /root/.cache/pip/wheels/93/08/0c/1b9866c35c8d3f136d100dfe88036a32e0795437daca089f70
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50467 sha256=322036c1e803de49dcf059ff1d41be023cd4bd3b12a28729f90ba290264287b7
  Stored in directory: /root/.cache/pip/wheels/38/f7/65/161985e7311dd484a23b3a5c9149995db

<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


62
Elapsed time is 44.354846 seconds.
None
63
Elapsed time is 31.620188 seconds.
None
64
Elapsed time is 38.736078 seconds.
None
65
Elapsed time is 31.141538 seconds.
None
66
Elapsed time is 31.166369 seconds.
None
67
Elapsed time is 39.773409 seconds.
None
68
Elapsed time is 37.534686 seconds.
None
69
Elapsed time is 97.446091 seconds.
None
70
Elapsed time is 36.136486 seconds.
None
71
Elapsed time is 38.983318 seconds.
None
72
Elapsed time is 33.956987 seconds.
None
73
Elapsed time is 38.843149 seconds.
None
74
Elapsed time is 45.316047 seconds.
None
75
Elapsed time is 30.180827 seconds.
None
76
Elapsed time is 44.778975 seconds.
None
77
Elapsed time is 34.063953 seconds.
None
78
Elapsed time is 39.079066 seconds.
None
79
Elapsed time is 33.379308 seconds.
None
80
Elapsed time is 37.533294 seconds.
None
81
Elapsed time is 39.645348 seconds.
None
82
Elapsed time is 45.781759 seconds.
None
83
Elapsed time is 40.348074 seconds.
None
84
Elapsed time is 35.169886 seconds.
None
85
Elapsed 

<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


202
Elapsed time is 43.556741 seconds.
None
203
Elapsed time is 44.970474 seconds.
None
204
Elapsed time is 36.413693 seconds.
None
205
Elapsed time is 47.396463 seconds.
None
206
Elapsed time is 31.392415 seconds.
None
207
Elapsed time is 41.382104 seconds.
None


...

415
Elapsed time is 58.757848 seconds.
None
416
Elapsed time is 59.908841 seconds.
None
417
Elapsed time is 123.648782 seconds.
None
418
Elapsed time is 59.341910 seconds.
None
419
Elapsed time is 60.513294 seconds.
None
420
Elapsed time is 61.960217 seconds.
None
421
Elapsed time is 60.806894 seconds.
None
422
Elapsed time is 70.216266 seconds.
None
423
Elapsed time is 70.660197 seconds.
None
424
Elapsed time is 69.815596 seconds.
None
425
Elapsed time is 56.240125 seconds.
None
426
Elapsed time is 66.715773 seconds.
None
427
Elapsed time is 71.589388 seconds.
None
428
Elapsed time is 64.280875 seconds.
None
429
Elapsed time is 84.186131 seconds.
None
430
Elapsed time is 41.902117 seconds.
None
431
Elapsed time is 75.410607 seconds.
None
432
Elapsed time is 34.412824 seconds.
None
433
Elapsed time is 37.161164 seconds.
None
434
Elapsed time is 40.878044 seconds.
None
435
Elapsed time is 42.091942 seconds.
None
436
Elapsed time is 35.658660 seconds.
None


<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


437
Elapsed time is 40.316303 seconds.
None
438
Elapsed time is 44.027266 seconds.
None
439
Elapsed time is 43.252526 seconds.
None
440
Elapsed time is 42.273967 seconds.
None


<string>:6: RuntimeWarning: Mean of empty slice
<string>:6: RuntimeWarning: Degrees of freedom <= 0 for slice.


441
Elapsed time is 31.725923 seconds.
None
442
Elapsed time is 41.780447 seconds.
None
443
Elapsed time is 40.443809 seconds.
None
444
Elapsed time is 41.157212 seconds.
None
445
Elapsed time is 34.986860 seconds.
None
446
Elapsed time is 92.475169 seconds.
None
447
Elapsed time is 61.538906 seconds.
None
448
Elapsed time is 67.860741 seconds.
None
449
Elapsed time is 33.916869 seconds.
None
450
Elapsed time is 41.302895 seconds.
None
451
Elapsed time is 109.901173 seconds.
None
452
Elapsed time is 47.018882 seconds.
None
453
Elapsed time is 40.563550 seconds.
None
454
Elapsed time is 49.254847 seconds.
None
455
Elapsed time is 41.077817 seconds.
None
456
Elapsed time is 41.062123 seconds.
None
457
Elapsed time is 50.250204 seconds.
None
458
Elapsed time is 42.642507 seconds.
None
459
Elapsed time is 43.275647 seconds.
None
460
Elapsed time is 43.675071 seconds.
None
461
Elapsed time is 42.509679 seconds.
None
462
Elapsed time is 34.280383 seconds.
None
463
Elapsed time is 43.626332 s

In [5]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
path = '/content/drive/My Drive/GSOC/WEEK 7/FINALoutput0_1000.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df1.to_csv(f)

Mounted at /content/drive


In [6]:
df1

,0,1,2,3,4,5,6,7,8,9,...,212841,212842,212843,212844,212845,212846,212847,212848,212849,target
0,0.919597,0.760238,1.101303,1.034527,0.837521,0.685986,0.688989,1.135959,1.092625,1.003870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1.025398,1.016231,0.987951,1.038180,1.023081,1.001104,0.982287,1.011871,1.004704,1.016585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1.071332,1.341602,1.092280,0.944699,0.984180,0.948657,0.896450,0.934901,0.752725,0.828430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1.017824,1.004170,0.850218,1.162286,0.989298,0.738813,1.040777,1.142984,0.836422,1.045461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,0.939003,0.839082,1.148021,0.913888,0.880893,1.153466,0.585936,0.759167,0.643355,0.761305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
982,0.589711,1.038403,0.661607,1.456011,1.044161,1.346148,0.518640,1.113973,1.243550,1.054374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
983,1.279825,0.679833,1.039892,0.624641,1.049217,1.175793,0.704409,0.849909,0.844958,1.115324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
984,0.944225,0.903859,0.957781,1.069567,0.796505,1.001235,1.045407,1.054819,0.818677,0.770443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
